### 1. **SimBench – Standardized Power System Benchmark**

#### Summary

| Number of Profiles | Profile Types                         | Load Details                                      | Renewables                  | Environment | Economy | Processed | Synthetic | Horizon     | Time Resolution | Geographical   |
|-----------|----------------------------------------|--------------------------------------------------|-----------------------------|------|-------|-----------|-----------|-------------|-----------|----------|
| 614       | Load, Renewable, Powerplant, Storage         | Active, Reactive, Residential, Industry, Commercial | Solar, Wind, Biomass, Hydro |  None  |  None   | True        | True        | 2016–2017   | 15 min    | Germany  |

---

#### Data Components:
The SimBench dataset includes four main time series profiles, each stored as a CSV file. These represent various actors in the power system and their corresponding roles.
> **Note:** There is no missing data.

| Profile             | Description                                            | Type         | Role     |
|---------------------|--------------------------------------------------------|--------------|----------|
| `LoadProfile`       (192 profiles)   | Consumer energy demand (Active, Reactive, Residential, Industry, Commercial)| Time series  | Consumer |
| `RESProfile`        (42 profiles)    | Renewable generation (Solar, Wind, Biomass, Hydro)     | Time series  | Producer |
| `PowerPlantProfile` (320 profiles)   | Powerplant generator output (e.g., powerplants)        | Time series  | Producer |
| `StorageProfile`    (60 profiles)    | Storage system charge/discharge behavior               | Time series  | Storage  |

---

#### File Locations:

| Profile             | Raw Folder Location                                         | Processed Folder Location                                        |
|---------------------|-----------------------------------------------------------|--------------------------------------------------------------------|
| `LoadProfile`       | [Consumer](./raw/consumer/)                               | [Consumer](./processed/consumer/)                     |
| `RESProfile`        | [Producer](./raw/producer/)                               | [Producer](./processed/producer/)                     |
| `PowerPlantProfile` | [Producer](./raw/producer/)                               | [Producer](./processed/producer/)                     |
| `StorageProfile`    | [Storage](./raw/1-complete_data-mixed-all-2-sw/producer/) |                      |

---

#### Profile Volume:  
- **Time Resolution:** 15 minutes  
- **Points per Day:** 96 (15-minute intervals × 24 hours)  
- **Total Length:** 35,136  
- **Total Duration Covered:** 366 days (1 full leap year)  
- **Scenarios:** 3 (same profiles reused in all)
> **Note:** All scenarios share the **same** time series profiles for **load and generation**.  
> The variation between scenarios lies in the network topology or component configurations, not in the profile data itself.
 

---

#### Description:  
SimBench is an open dataset of standardized electrical power system benchmark models from low (LV) to extra-high (EHV) voltage grids. It includes synchronized grid topology and full-year time series profiles for consumer demand, renewable and conventional generation, and energy storage systems. Designed for direct compatibility with the `pandapower` Python library.

> **Note**: All time series profiles in SimBench are **synthetic**, not based on real-world measurements.  
> *“For DERs, synthetic time series are provided in the SimBench dataset.”*  
> — [SimBench Paper, MDPI Energies (2020)](https://www.mdpi.com/1996-1073/13/12/3290)

> **Units:**  
> - **Active power** (load/generation): **MW**  
> - **Reactive power**: **MVAr**  
> - **Voltage**: **kV**  
> - **Storage power capacity**: **MW**  
> - **Storage energy capacity**: **MWh**
---

#### Related Links:  
  - [GitHub Repository](https://github.com/e2nIEE/simbench)
  - [SimBench Paper (MDPI Energies, 2020)](https://www.mdpi.com/1996-1073/13/12/3290)
  - [SimBench Documentation](https://simbench.readthedocs.io/en/stable/)


In [ ]:
import pandas as pd

# Load CSVs using updated structure
load_profile = pd.read_csv("raw/consumer/LoadProfile.csv", sep=";")
res_profile = pd.read_csv("raw/producer/RESProfile.csv", sep=";")
powerplant_profile = pd.read_csv("raw/producer/PowerPlantProfile.csv", sep=";")


In [ ]:
# Preview of LoadProfile
load_profile.head(100)

In [ ]:
# Preview of RESProfile
res_profile.head(100)

In [ ]:
# Preview of PowerPlantProfile
powerplant_profile.head(100)

In [ ]:
# Length of each data profile in a table
profiles = {
    "LoadProfile": load_profile,
    "RESProfile": res_profile,
    "PowerPlantProfile": powerplant_profile
}

lengths = {name: len(df) for name, df in profiles.items()}
length_df = pd.DataFrame.from_dict(lengths, orient='index', columns=['Length'])
print(length_df)

In [ ]:
# Check if there is any missing data
for name, df in profiles.items():
    print(f"\nMissing values in {name}:")
    print(df.isna().sum())

In [ ]:
# Convert time column to datetime for all profiles
for df in [load_profile, res_profile, powerplant_profile]:
    df['time'] = pd.to_datetime(df['time'], dayfirst=True)

In [ ]:
# Check if there are any missing time stamps
for name, df in profiles.items():
    expected_times = pd.date_range(start=df['time'].min(), end=df['time'].max(), freq='15min')
    actual_times = df['time']
    missing_times = expected_times.difference(actual_times)

    print(f"\n{name} missing timestamps: {len(missing_times)}")
    if not missing_times.empty:
        print(missing_times)

# 1 Missing time stamp from daylight saving time


In [ ]:
import sys
sys.path.append("..")

from utils.simbench_plotter import (
    plot_simbench_total_load_over_time,
    plot_simbench_max_energy_by_category,
)

fp_load = "raw/consumer/LoadProfile.csv"
fp_res  = "raw/producer/RESProfile.csv"

# Active/Reactive load over time period, downsampled to 300 points
plot_simbench_total_load_over_time(
    fp=fp_load,
    kind="pload",  # or "qload" for reactive
    start_time="2016-01-01 00:00:00",
    end_time="2016-01-02 00:00:00",
    max_points=300
)

# Max energy for each renewable profile within the time period, downsampled to 300 points (i.e. the result could vary slightly)
plot_simbench_max_energy_by_category(
    fp_res,
    start_time="2016-01-01 00:00:00",
    end_time="2016-01-02 00:00:00",
    max_points=300
)


In [ ]:
# Calculate the number of profiles in each profile category
load_profiles = pd.read_csv("raw/consumer/LoadProfile.csv", sep=";")

# Total number of unique profile columns (excluding time columns)
num_profiles = (
    len(load_profiles.columns) - 1
)

print(f"Load profile contains {num_profiles} profiles")


In [ ]:
res_profiles = pd.read_csv("raw/producer/RESProfile.csv", sep=";")

# Total number of unique profile columns (excluding time columns)
num_profiles = (
    len(res_profiles.columns) - 1 
)

print(f"Renewable profile contains {num_profiles} profiles")


In [ ]:
pp_profiles = pd.read_csv("raw/producer/PowerPlantProfile.csv", sep=";")

# Total number of unique profile columns (excluding time columns)
num_profiles = (
    len(pp_profiles.columns) - 1 
)

print(f"Power plant profile contains {num_profiles} profiles")


In [ ]:
st_profiles = pd.read_csv("raw/1-complete_data-mixed-all-2-sw/StorageProfile.csv", sep=";")

# Total number of unique profile columns (excluding time columns)
num_profiles = (
    len(st_profiles.columns) - 1 
)

print(f"Storage profile contains {num_profiles} profiles")
